In [1]:
import pandas as pd
from datetime import datetime, timedelta
import util_calendar

In [2]:
import pygsheets
gc = pygsheets.authorize(client_secret = 'creds_google/credentials.json')

In [3]:
sh = gc.open_by_key('1GDXzzTD1dBnXWcpjPXqfIjpLwxHtXVQtjwfLhCaZNHA')

In [4]:
study_sheet = sh.worksheet_by_title("STUDY-PLAN")

In [27]:
df = study_sheet.get_as_df(start="A", end="G")
df

In [29]:
# Helpful Functions        
def calendar_dict(series):
    # Create a new calendar event from this event

    # Extract the Date, Time & Duration     
    start_date = datetime.strptime(series['Time']+ " " + series['Date'], "%I:%M %p %d-%b-%Y")
    duration = datetime.strptime(series['Duration'], "%H:%M")

    # Calculate the End Date
    end_date = start_date + timedelta(minutes = duration.minute, hours = duration.hour)

    # Convert to Text ISO Format
    start_date = datetime.isoformat(start_date, timespec='milliseconds')
    end_date = datetime.isoformat(end_date, timespec='milliseconds')


    

    
    tz = 'GMT+3:00'
    event_bluprnt = {
      'summary': series['Name'],
      'location': 'Virtual',
      'start': {
        'dateTime': start_date,
        'timeZone': tz
      },
      'end': {
        'dateTime': end_date,
          'timeZone': tz
      },
      
    }
    
    # Check to see if the keywords for Frequency is valid
    # [checking function here]
    freq = series["Frequency"].upper()
    if freq not in ["", "ONCE"]:
        event_bluprnt['recurrence'] = [ f'RRULE:FREQ={freq}' ]
    
    return event_bluprnt
    
    
def initiate_event(event_bluprnt):
    # Pubish the event
    cal_code  = util_calendar.insert_event(event_bluprnt)
    return cal_code
    


In [8]:
event = df.loc[0]
print(event)


Name                           IBM Data Science
Duration                                   0:25
Time                                    2:00 PM
Date                                22-Jan-2021
Frequency                                 Daily
Google Calendar Invite Code                 dfd
Update                                     TRUE
Name: 0, dtype: object


In [30]:
for index in range(len(df)):
    event = df.loc[index]
    if event['Google Calendar Invite Code'] == '':
        # Create New Event
        mandatory_entries = event[['Name', 'Duration', 'Time', 'Date']]
        if (mandatory_entries == "").any():
            print(f"Fields are missing for {event['Name']}")
        else:
            event_bluprnt = calendar_dict(event)
            print(event_bluprnt)
            event_code = initiate_event(event_bluprnt)
                  
            # Write back to the dataframe with the returned Invite Code
            df.loc[index,'Google Calendar Invite Code'] = event_code
                  
# Write back to the sheet
study_sheet.set_dataframe(df, start='A2', copy_head=False)

{'summary': 'IBM Data Science', 'location': 'Virtual', 'start': {'dateTime': '2021-01-22T14:00:00.000', 'timeZone': 'GMT+3:00'}, 'end': {'dateTime': '2021-01-22T14:25:00.000', 'timeZone': 'GMT+3:00'}}
Fields are missing for Course on Philosophy
